<a href="https://colab.research.google.com/github/vivivicdjdk/machine/blob/main/%EA%B3%BC%EC%A0%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#타이타닉

학습 데이터(train.csv): 승객의 생존 여부 정보

테스트 데이터(test.csv): 승객의 생존 여부 정보 x

이 정보가 빠진 테스트 데이터를 통해 승객의 생존 여부를 예측

##데이터 준비

### 데이터 입력

In [1]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/titanic/test.csv')

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
train.info()
print('-----------------------------------------')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
-----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Passenge

## 데이터 전처리

In [3]:
train_test_data = [train, test]

## Name(이름)

이름에서 호칭 추출

In [4]:
for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract('([\w]+)\.', expand=False)

In [5]:
print(pd.unique(train['Title']))

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer']


각 호칭들을 숫자에 매핑

In [6]:
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].apply(lambda x: 0 if x=="Mr" else 1 if x=="Miss" else 2 if x=="Mrs" else 3 if x=="Master" else 4)

### Age(나이)

In [7]:
for dataset in train_test_data:
  # Age의 결측치를 평균값으로 채움
  for dataset in train_test_data:
    dataset['Age'].fillna(dataset.groupby("Title")["Age"].transform("mean"), inplace=True)
  # Age의 범주를 5개로 지정
  dataset['Agebin'] = pd.cut(dataset['Age'], 5, labels=[0,1,2,3,4])

### Sex(성별)


In [8]:
sex_mapping = {'male': 0, 'female': 1}

for dataset in train_test_data:
  dataset['Sex'] = dataset['Sex'].map(sex_mapping)

### SibSp(형제자매)와 Parch(부모자녀)

In [9]:
for dataset in train_test_data:
  # 가족 수 = 형제자매 + 부모님 + 자녀 + 본인
  dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
  dataset['IsAlone'] =1
  # 동승자 여부 판단(가족 수 > 1이면 동승자 있음)
  dataset.loc[dataset['FamilySize']>1, 'IsAlone'] = 0

### Embarked(탑승 지역)

훈련 데이터의 Embarked의 결측치를 최빈값으로 채움

In [10]:
train['Embarked']=train['Embarked'].fillna(train['Embarked'].value_counts().index[0])

In [11]:
embarked_mapping = {'S':0, 'C':1, 'Q':2}

for dataset in train_test_data:
  dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)

### Fare(탑승 요금)

테스트세트의 Fare 결측치 행 삭제

In [12]:
for dataset in train_test_data:
  dataset['Farebin'] = pd.qcut(dataset['Fare'], 4, labels=[0,1,2,3])
test = test.dropna(subset=['Fare'], axis=0)

### Cabin(방 번호)

유실율이 높으므로 열 전체를 삭제

전처리가 끝났거나 훈련에 이용하지 않을 데이터 삭제

In [13]:
drop_column=['Name','Age','Fare','SibSp','Parch','Ticket','Cabin']
train = train.drop(drop_column, axis=1)
test = test.drop(drop_column, axis=1)

In [14]:
train.info()
print('-----------------------------------------')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Sex          891 non-null    int64   
 4   Embarked     891 non-null    int64   
 5   Title        891 non-null    int64   
 6   FamilySize   891 non-null    int64   
 7   IsAlone      891 non-null    int64   
 8   Farebin      891 non-null    category
dtypes: category(1), int64(8)
memory usage: 56.9 KB
-----------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 0 to 417
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  417 non-null    int64   
 1   Pclass       417 non-null    int64   
 2   Sex          417 non-null    int64   
 3   Embar

# 데이터 학습

### 입력데이터, 타겟데이터 생성


훈련 결과인 Survived는 triain_target에 저장

In [15]:
import numpy as np
# 훈련에 필요하지 않은 데이터 삭제
train_input = train.drop(['PassengerId', 'Survived'], axis=1).to_numpy()
train_target = train['Survived'].to_numpy()

test_input = test.drop(['PassengerId'], axis=1).to_numpy()

## 확률적 경사 하강법(SGD)

In [16]:
from sklearn.linear_model import SGDClassifier
sc = SGDClassifier(loss='log_loss', random_state=202035236) # 이진 분류 문제이므로 로지스틱 회귀 손실 함수 사용
sc.fit(train_input, train_target)
print(sc.score(train_input, train_target))

0.8058361391694725


In [17]:
# 교차 검증 점수 확인
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(sc, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.7621555457912246


### 최적화

확률적 경사 하강법의 하이퍼 파라미터 : max_iter

에포크 횟수를 결정

에포크 횟수가 너무 적으면 과소적합

에포크 횟수가 너무 많으면 과대적합이 발생한다.

In [18]:
from sklearn.model_selection import GridSearchCV
params = {'max_iter':range(50,1000,50)}
sgd_gs = GridSearchCV(sc, params, n_jobs=-1) # 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체 생성
sgd_gs.fit(train_input, train_target)
# 그리드서치로 찾은 최적의 매개변수
print(sgd_gs.best_params_)

{'max_iter': 100}


In [19]:
# 교차 검증의 최종 점수
print(np.mean(scores['test_score']))
# 찾은 최상의 매개변수 조합으로 모델 생성
sc = sgd_gs.best_estimator_
# 최적화 완료 후 훈련 세트 점수 확인
print(sc.score(train_input, train_target))

0.7621555457912246
0.8058361391694725


## 결정트리

In [20]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=202035236)
dt.fit(train_input, train_target)

print(dt.score(train_input, train_target))

0.8574635241301908


In [21]:
# 교차 검증 점수 확인
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.797991337643588


과대적합임

### 최적화

min_impurity_decrease : 노드의 정보이득이 매개변수보다 작아지면 더이상 분할하지 않음(과대적합 방지)

max deapth : 트리의 최대 깊이 지정

min_samples_split : 노드를 분할하기 위한 최소 샘플의 수(기본값 2) 작게 설정할 수록 분할 노드가 많아져 과적합 가능성 증가

min_smples_leaf : 리프 노드가 되기 위한 최소 샘플의 수

In [22]:
from scipy.stats import uniform, randint

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(10, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }
from sklearn.model_selection import RandomizedSearchCV
dt = DecisionTreeClassifier(random_state=202035236)
dt_rs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=202035236), params,
                        n_iter=100, n_jobs=-1, random_state=202035236)
#n_iter 반복횟수 지정 (이 범위 내에서 100번 선택하겠다.)
dt_rs.fit(train_input, train_target)
print(dt_rs.best_params_)

{'max_depth': 23, 'min_impurity_decrease': 0.0004483185889588648, 'min_samples_leaf': 12, 'min_samples_split': 15}


In [23]:
# 최상의 교차 검증 점수 확인
print(np.max(dt_rs.cv_results_['mean_test_score']))
# 찾은 파라미터로 모델을 생성하고 훈련 점수 확인
dt = dt_rs.best_estimator_
print(dt.score(train_input, train_target))

0.8305002824681438
0.8338945005611672


## 랜덤포레스트

In [24]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=202035236)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 훈련 점수, 검증 점수

0.8641978032368376 0.8036030381018142


### 최적화

In [25]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(10, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

rf = RandomForestClassifier(random_state=202035236)
rf_rs = RandomizedSearchCV(RandomForestClassifier(random_state=202035236), params,
                        n_iter=100, n_jobs=-1, random_state=202035236)
#n_iter 반복횟수 지정 (이 범위 내에서 100번 선택하겠다.)
rf_rs.fit(train_input, train_target)
print(rf_rs.best_params_)

{'max_depth': 16, 'min_impurity_decrease': 0.0008497651568504537, 'min_samples_leaf': 5, 'min_samples_split': 12}


In [26]:
# 최상의 교차 검증 점수 확인
print(np.max(rf_rs.cv_results_['mean_test_score']))
# 찾은 파라미터로 모델을 생성하고 훈련 점수 확인
rf = rf_rs.best_estimator_
print(rf.score(train_input, train_target))

0.8237775406440273
0.835016835016835


## 히스토그램기반 그레이디언트 부스팅

In [27]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=202035236)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))  # 훈련 점수, 검증 점수

0.8552192823486771 0.8092021844203126


### 최적화

learning_rate : 학습을 진행할 때마다 적용하는 학습률(기본값 0.1)

max_iter : 부스팅 반복 횟수(n_estimator 대신 사용)

In [28]:
params = {'learning_rate': np.arange(0.01, 0.1, 0.01),
          'max_iter' : range(100, 500, 50)
          }

hgb_gs = GridSearchCV(HistGradientBoostingClassifier(random_state=20203536), params, n_jobs=-1)
hgb_gs.fit(train_input, train_target)
print(hgb_gs.best_params_)

{'learning_rate': 0.01, 'max_iter': 100}


In [29]:
# 최상의 교차 검증 점수 확인
print(np.max(hgb_gs.cv_results_['mean_test_score']))
# 찾은 파라미터로 모델을 생성하고 훈련 점수 확인
hgb = hgb_gs.best_estimator_
print(hgb.score(train_input, train_target))

0.8294017952419811
0.8383838383838383
